In [1]:
import string
import re
import os
import tempfile
import logging
import csv
from datetime import datetime
from gensim import corpora
from gensim import models
from gensim.corpora import Dictionary
from collections import namedtuple
import gensim.parsing.preprocessing as processing
from os import listdir
import CustomApi as api
#Start Global Variables and Types
#document dir
docs = "../data/documents/"

#Number of topics the model should index
numberOfTopics = 125
#Number of passes the model should make
passes = 10
#MyAbstract(title:String, date:datetime.datetime, text:String)
MyAbstract = namedtuple('MyAbstract',["title","date","text"])
#End Global Variables and Types

C:\Users\USER\Anaconda3\envs\py27\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
files = listdir(docs+"lt2006/")
entries = []

In [3]:
for file in files:
    with open(docs+"lt2006/"+file,"rb") as csvfile:
        spamreader = csv.reader(csvfile)
        for row in spamreader:
            if "Authors" not in row[0] and row[15] !='[No abstract available]':
                entries.append(row)
        

In [4]:
entries[0][1]


'Evidence of the Juvenile Hormone Methyl(2E,6E)-10,11-epoxy-3,7,11-trimethyl-2,6-dodecadienoate(JH-3) in Insects of Four Orders'

In [5]:
# Create a set of frequent words
stopFile = open("../data/stopwords.txt","r")
stopWords = stopFile.read().splitlines()
stopWords.append("\xc2\xa9") #This is the copyright symbol, this shows up in every abstract and should not be apart of the corpus
stopWords.extend(["\u2019","\u03bc","bee","bees","honey","honeybee","honeybees"])
stopList = set(stopWords)
with open("../data/extraStopWords.txt","r") as extraStopFile:
    stopWords.extend(extraStopFile.read().split("\n"))
# Lowercase each document, split it by white space and filter out stopWords
processing.STOPWORDS = stopWords
def removeStops(text):
    return processing.stem_text(processing.remove_stopwords(text.lower().translate(None, string.punctuation)))
# Each abstract has a 'title':String, 'date':datetime.datetime, and 'text':String
abstracts = [MyAbstract._make([art[1],datetime.strptime(art[2], '%Y'),removeStops(art[15])]) for art in entries]
abstracts.sort(key=lambda q: q.date.year)
entries = None
# Count word frequencies
selections = []

access = lambda x: x.date.year
lastIndex = 0
for i in range(1957,2007):
    index = api.BinarySearch(abstracts,i,access)
    if  index != -1:
        selections.append(abstracts[lastIndex:index+1])
        lastIndex = index+1
        


C:\Users\USER\Anaconda3\envs\py27\lib\site-packages\gensim\parsing\preprocessing.py:105: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return " ".join(w for w in s.split() if w not in STOPWORDS)


In [ ]:
from collections import defaultdict
def createCorpus(selection):
    frequency = defaultdict(int)
    for abst in abstracts:
        for token in abst.text.split(" "):
            frequency[token] += 1
    tempFolder = tempfile.gettempdir()
    dictionary = corpora.Dictionary([[token for token in abst.text.split(" ") if frequency[token] > 5] for abst in selection])
    dictionary.save(os.path.join(tempFolder,'words.dict'))
    # Create general corpus and serialize in order for it to be iterated over
    corpus = [dictionary.doc2bow(text) for text in processedCorpus]
    corpora.MmCorpus.serialize(os.path.join(tempFolder, 'words.dict'), corpus)
    


# Save the dictionary of tokens
# def createModel():
#     tempFolder = tempfile.gettempdir()
#     dictionary = corpora.Dictionary(processedCorpusFor(selection))
#     dictionary.save(os.path.join(tempFolder,'words.dict'))
#     # Create general corpus and serialize in order for it to be iterated over
#     corpus = [dictionary.doc2bow(text) for text in processedCorpus]
#     corpora.MmCorpus.serialize(os.path.join(tempFolder, 'words.dict'), corpus)
    # Train the model and set number of topics
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
def writeOutTopWords
#Saves the top 5 topics and top 3 words per topic
with open("../data/Docbow/Top5TopicsTop3WordsTop="+str(numberOfTopics)+".tsv","w") as file:
    docs = [dictionary.doc2bow(a.split()) for a in abstracts] # Convert to bag of words format first
    for d in xrange(0,len(docs)):
        topics = lda.get_document_topics(bow=docs[d])
        topics = sorted(topics,key=lambda k:k[1], reverse=True)
        file.write(yearOfAbstract[d]+":\t")
        for t in topics[0:4]:
            for word in [dictionary[x[0]] for x in lda.get_topic_terms(t[0],topn=3)]:
                file.write(word+"\t")
        file.write("\n")

In [14]:
size = 0
start = 0
for i in range(0,len(selections)):
    size += len(selections[i])
    if size >=50:
        corpus = createCorpus(api.flaten(selections[start:i+1]))
        lda = models.ldamodel.LdaModel(corpus,id2word=dictionary,num_topics=numberOfTopics)
        topics = lda.top_topics(corpus=corpus)
        size = 0
        start = i+1

58
51
69
54
70
55
83
79
73
72
90
85
85
74
99
144
159
148
134
142
159
201
238
238
271
260
299
303
401
453
451
446


The above corpus shows the amount of times every word used in the documents is used in every indevidual document. Every word is represented by a token ID, the list of which can be found in "words.dict"

In [ ]:
# Sort the most interesting words per topic per document
# This cell does not need to be run if only trying to create Top Nine terms per paper
topicOrganizingFile = open("../data/topicorganization.tsv","w")
for x in xrange(0,len(abstracts)):
    doc = dictionary.doc2bow(abstracts[x].split())
    docTopics, wordTopics, phiValues = lda.get_document_topics(doc, per_word_topics=True)
    topicOrganizingFile.write(yearOfAbstract[x]+"\t"+titleOfAbstract[x]+"\t")
    for y in xrange(0,min(3,len(docTopics))):
        topicnumber = docTopics[y][0]
        topicOrganizingFile.write(str(lda.show_topic(topicnumber))+"\t")
        #Sorts the word topics in decending order based on their greatest phi value
        for z in xrange(0,len(phiValues)):
            phiValues[z][1].sort(key=lambda q:q[1],reverse=True)
        phiValues.sort(key=lambda q:q[1][0][1],reverse=True)
        curindex=0
        topwords = ""
        for z in xrange(0,3):
            while curindex<len(phiValues) and phiValues[curindex][1][0][0]!=topicnumber:
                curindex+=1
            if(curindex>=len(phiValues)):break
            print len(phiValues)
            print dictionary[phiValues[curindex][0]]
            topwords+=str(dictionary[phiValues[curindex][0]].encode('utf-8').strip())+" "
            curindex+=1
        filter(lambda a:a[0]!=topicnumber,phiValues)
        topicOrganizingFile.write(topwords+"\t")
    topicOrganizingFile.write("\n")
topicOrganizingFile.close()

        

In [ ]:
topicWords = []
for i in range(0,numberOfTopics):
    t = lda.get_topic_terms(i,50)
    currentWordList = []
    for x in t:
        word = str(dictionary[x[0]])
        if word not in currentWordList:
            currentWordList.append(word)
    topicWords.append(currentWordList)
topicListFile = open("../data/TopicWords/List-"+str(numberOfTopics)+".txt","w+")
for i in range(0,len(topicWords)):
    topicListFile.write("Topic "+str(i)+":\n")
    for j in topicWords[i]:
        topicListFile.write(j+'\n')
    topicListFile.write('\n')
topicListFile.close()

In [ ]:
#Makes the top nine terms for each document

topNineFile = open("../data/Docbow/TopNineTerms-"+str(numberOfTopics)+".tsv","w")
for abstr in abstracts:
    doc = dictionary.doc2bow(abstracts[2].split()) # Convert to bag of words format first
    # Get the topics and words associated with each document
    docTopics, wordTopics, phiValues = lda.get_document_topics(doc, per_word_topics=True)
    topNineFile.write(yearOfAbstract[x]+"\t"+abst+"\t")
    for z in xrange(0,len(phiValues)):
        phiValues[z][1].sort(key=lambda q:q[1],reverse=True)
    phiValues.sort(key=lambda q:q[1][0][1],reverse=True)
    nineWords = ""
    for x in phiValues[:15]:
        nineWords+= dictionary[x[0]] + " "
    topNineFile.write(nineWords.encode('utf-8')+"\n")

In [37]:

access = lambda x: x.date.year
api.BinarySearch(abstracts,term=1971,accessElement=access)


152

In [23]:
abstracts[0]

MyAbstract(title='The food of adult drone honeybees (Apis mellifera)', date=datetime.datetime(1957, 1, 1, 0, 0), text=u'summaryo1dron behaviour insid hive beenstudi gener observ numer mark drone continu observ particular drone predetermin period day2dron spend nearli threequart theirtim whilst insid hive period appar inact frequent broken period movement combs3for first dai live dronesar fed entir worker follow period fed worker although extent previous feed cell week old tend feed entir beg workers4work 226 dai old seen tofe drone although 46 dai old activ probabl worker feed drone brood food5th behaviour worker toward drone appear depend upon ag older drone attack worker younger drone still fed6th worker attack drone probabl unemploi forag \xa9 1957')

In [16]:
selections[49]

[MyAbstract(title='Role of histamine as a putative inhibitory transmitter in the honeybee antennal lobe', date=datetime.datetime(2006, 1, 1, 0, 0), text=u'background odors represented specific spatiotemporal patterns olfactory bulb vertebrates analogue antennal lobe inhibitory circuits al involved processing odors shape afferent odor responses gaba known inhibitory transmitter antennal lobe interneurons gabaergic therefore sought analyze functional inhibitory transmitter histamine processing odors al optically recorded representation odors histamine application input estimated compound signal output selectively measuring projection neurons histamine led strong reversible reduction odorevoked responses conclusion propose histamine addition gaba acts inhibitory transmitter al therefore likely play odor processing \xa9 2006 sachse et al licensee biomed central ltd'),
 MyAbstract(title='Spatial distribution patterns of Acute Bee Paralysis Virus, Black Queen Cell Virus and Sacbrood Virus in